In [15]:
import os
from pathlib import Path
from mubelnet.utils import holdout_split
from jax import random
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from mubelnet.utils import perplexity
from statkit.non_parametric import bootstrap_score
from GenomeSigInfer.sbs import SBSMatrixGenerator

In [16]:
_PATH = Path(os.getcwd()).parent.parent/ "data" / "vcf"
files = (_PATH / "WES_Other.20180327.simple", _PATH / "WGS_Other.20180413.simple")
_NUCL_MAP = {
    "nucl_strength": {"A": "W", "T": "W", "C": "S", "G": "S"},
    "amino": {"A": "M", "C": "M", "G": "K", "T": "K"},
    "structure": {"A": "R", "C": "Y", "G": "R", "T": "Y"},
}

In [17]:
%%script echo skipping
# This code was ran on the cluster due to RAM limitations

def cluster_df(df: pd.DataFrame, cluster: dict[str, str], cluster_type: str) -> None:
    """
    Clusters the mutation types in the given DataFrame based on the provided cluster dictionary.
    Saves the clustered DataFrame as a parquet file.

    Args:
        df (pd.DataFrame): The DataFrame containing mutation data.
        cluster (dict[str, str]): A dictionary mapping individual characters to their corresponding cluster.
        cluster_type (str): The type of the DataFrame.

    Returns:
        None
    """
    df_copy = df.copy()
    df_copy["MutationType"] = df_copy["MutationType"].apply(
        lambda x: "".join(cluster.get(c, c) for c in x[:2])
        + x[2:-2]
        + "".join(cluster.get(c, c) for c in x[-2:])
    )
    df_copy = df_copy.groupby("MutationType").sum()
    filename = (
        Path(__file__).parent.parent
        / "sbs"
        / f"sbs.{df_copy.shape[0]}.{cluster_type}.parquet"
    )
    df_copy.to_parquet(filename)

sbs_9_context = SBSMatrixGenerator.generate_single_sbs_matrix(
    "./analyses/SBS/",
    files,
    "./analyses/ref_genome/GRCh37",
    "GRCh37",
    max_context=9,
)
sbs_9_context.to_parquet(
    Path(__file__).parent.parent / "sbs" / f"sbs.{sbs_9_context.shape[0]}.parquet"
)

for cluster_type in _NUCL_MAP:
    cluster_df(sbs_9_context, _NUCL_MAP[cluster_type], cluster_type)

Couldn't find program: 'echo'


In [22]:
def calculate_bootstrap_score(all_genomes, signatures, nmf_init, beta_loss):
    X_train, X_test = holdout_split(random.PRNGKey(42), all_genomes)
    nmf = NMF(
        n_components=signatures,
        init=nmf_init,
        beta_loss=beta_loss,
        solver="cd",
        max_iter=10_000,
        tol=1e-15,
    ).fit(X_train)
    h = nmf.transform(X_train)
    unnormed_probs = h @ nmf.components_
    probs_nmf = unnormed_probs / unnormed_probs.sum(axis=-1, keepdims=True)
    # Compute performance on test set.
    is_inf = (probs_nmf == 0) & (X_test > 0)
    # Delete zero rows.
    zero_rows = np.where(is_inf)[0]
    print('Deleting rows', zero_rows)
    X_test_subset = np.delete(X_test, zero_rows, axis=0)
    probs_sigprof_xtrct_subset = np.delete(probs_nmf, zero_rows, axis=0)

    pp_nmf = bootstrap_score(X_test_subset, probs_sigprof_xtrct_subset, metric=perplexity, random_state=43)
    print('Perplexity on unseen observations (removed samples causing infinite perplexity)', pp_nmf)
    print(pp_nmf.latex())
    return pp_nmf

def calculate_perplexity(filename: str | Path, signatures: int, init: None | str, beta_loss: str):
    sbs_file = _PATH.parent.parent / "analyses" / "SBS" / filename
    df = pd.read_parquet(sbs_file)
    all_genomes = np.array(df.iloc[:, 1:])
    pp_nmf = calculate_bootstrap_score(all_genomes, signatures, init, beta_loss)
    print(pp_nmf)

In [ ]:
file = "sbs.96.parquet"
calculate_perplexity(file, 48, None, "frobenius")

In [ ]:
file = "sbs.1536.parquet"
calculate_perplexity(file, 48, None, "frobenius")

In [ ]:
file = "sbs.24576.parquet"
calculate_perplexity(file, 48, None, "frobenius")

In [ ]:
file = "sbs.24576.amino.parquett"
calculate_perplexity(file, 48, None, "frobenius")

In [ ]:
file = "sbs.24576.nucl_strength.parquet"
calculate_perplexity(file, 48, None, "frobenius")

In [ ]:
file = "sbs.24576.structure.parquet"
calculate_perplexity(file, 48, None, "frobenius")